# Minsearch to search context in documents

In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields = {'question','text','section'},
    keyword_fields=['course']
)

In [7]:
q = 'the course already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {'question':3.0, 'section':0.5}
results = index.search(
    query=q,
    boost_dict = boost,
    filter_dict = {'couse':'data-engineering-zoomcamp'},
    num_results=5
)

# OpenAI 

In [10]:
from openai import OpenAI

In [11]:
import sys,os,os.path

In [12]:
api_key = os.environ['OPEN_AI_API_KEY']

In [13]:
client = OpenAI(api_key=api_key)

In [14]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-16k',
    messages = [{'role':'user',"content":"is it to late join the course"}]
)

In [15]:
response.choices[0].message.content

'It depends on the course and the institution offering it. Some courses may have specific enrollment deadlines, while others may allow for late entry. It is best to contact the course coordinator or administration to inquire about late enrollment options.'

In [16]:
prompt_template = """
You are Course Assistant. You answer the question based on context. Use only fact the context for answering the question.

QUESTION: {question}
CONTEXT : 
{context}
""".strip()

In [17]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

In [18]:
prompt = prompt_template.format(question=q,context=context).strip()

In [19]:
print(prompt)

You are Course Assistant. You answer the question based on context. Use only fact the context for answering the question.

QUESTION: the course already started, can I still enroll?
CONTEXT : 
section: General course-related questions
question:The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last mi

In [20]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo-16k',
    messages = [{'role':'user',"content":prompt}]
)

In [21]:
print(response.choices[0].message.content)

Based on the given context, if the course has already started, it is still possible to join the course. However, if joining late, some homework submissions may not be allowed. To be eligible for a certificate, it is necessary to submit 2 out of 3 course projects and review 3 peers' projects by the deadline. Even if not registered, it is still possible to submit the homeworks, but be aware of the deadlines for turning in the final projects. After the course finishes, all materials will still be available, allowing learners to follow the course at their own pace. Before the course starts, it is recommended to install and set up the necessary dependencies and requirements. The exact start date of the course is January 15, 2024, at 17:00.


# Mistral AI API

In [22]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [23]:
mistral_api_key = os.environ["MISTRAL_AI_API_KEY"]

In [24]:
client = MistralClient(api_key=mistral_api_key)
model = "mistral-large-latest"
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content=prompt)]
)

print(chat_response.choices[0].message.content)

Yes, you can still enroll in the course even if it has already started. You may not be able to submit some of the homeworks, but you can still participate in the course. To be eligible for a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline. If you join the course now and manage to complete these requirements, you will still be eligible for a certificate.


In [25]:
def search(query):
    boost = {'question':3.0, 'section':0.5}
    results = index.search(
        query=query,
        boost_dict = boost,
        filter_dict = {'couse':'data-engineering-zoomcamp'},
        num_results=5
    )
    return results

In [26]:
def build_prompt(query,search_results):
    prompt_template = """
    You are Course Assistant. You answer the question based on context. Use only fact the CONTEXT for answering the question.
    
    QUESTION: {question}
    CONTEXT : 
    {context}
    """.strip()
    context = ""
    for doc in results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query,context=search_results).strip()
    return prompt

In [27]:
def llm_mistral(prompt):
    client = MistralClient(api_key=mistral_api_key)
    model = "mistral-large-latest"
    chat_response = client.chat(
        model=model,
        messages=[ChatMessage(role="user", content=prompt)])
    return chat_response.choices[0].message.content

In [28]:
def llm_open_api(prompt):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(model='gpt-3.5-turbo-16k',messages = [{'role':'user',"content":prompt}])
    return response.choices[0].message.content

In [29]:
query = 'the course already started. Can I still enroll the course?'
search_results = search(query)
prompt = build_prompt(query,search_results)
answer = llm_mistral(prompt)
print(answer)

Yes, you can still enroll in the course even if it has already started. However, you may not be able to submit some of the homeworks. To be eligible for a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline. If you join the course later and manage to complete these requirements, you will still be eligible for a certificate.


In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

# ElasticSearch

In [31]:
from elasticsearch import Elasticsearch

In [32]:
es_client = Elasticsearch('http://127.0.0.1:9200')

In [33]:
es_client.info()

ObjectApiResponse({'name': 'da099086a704', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KWEVtxkKSp-J3ZzcH-_0Yg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [34]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course_question'
es_client.indices.create(index=index_name,body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_question'})

In [35]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

In [38]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [39]:
response = es_client.search(index=index_name,body=search_query)

In [40]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [41]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [42]:
def elastic_query(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name,body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [43]:
query = 'the course already started. Can I still enroll the course?'
elastic_query(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [44]:
def rag(query):
    search_results = elastic_query(query)
    prompt = build_prompt(query,search_results)
    answer = llm_mistral(prompt)
    return answer

In [45]:
rag(query)

"Yes, you can still enroll in the course even if it has already started. You're eligible to submit homeworks, but please be aware that there will be deadlines for turning in the final projects. It's recommended not to leave everything for the last minute."

# Homework

In [50]:
index_name = 'course_question'
query= "How do I execute a command in a running docker container?"
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
response = es_client.search(index=index_name,body=search_query)

In [51]:
score_results = []
for score in response['hits']['hits']:
    score_results.append(score['_score'])
score_results 

[84.050095]

In [55]:
query = "How do I execute a command in a running docker container?"
index_name = 'course_question'
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name,body=search_query)

In [56]:
list_question = []
for hit in response['hits']['hits']:
    print(hit['_source']['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


In [59]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [67]:
result_docs[1]['text']

"You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani"

In [71]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [57]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [58]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [97]:
result_docs = response['hits']['hits']
formatted_docs = [context_template.format(question=doc['_source']['question'], text=doc['_source']['text']) for doc in result_docs]

In [98]:
context_result = []
for i in formatted_docs:
    context_result.append(i)

In [99]:
context_result = '\n'.join(context_result)

In [100]:
prompt = prompt_template.format(question=question,context=context_result).strip()

In [104]:
len(prompt)

1460

In [105]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.8 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 18.5 MB/s eta 0:00:00m eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [107]:
import tiktoken

In [108]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [112]:
len(encoding.encode(prompt))

322

In [113]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [114]:
llm_open_api(prompt)

"A: To execute a command in a running Docker container, you can use the `docker exec` command. First, use `docker ps` to find the container ID. Then, use the `docker exec` command with the container ID and the command you want to execute. Here's an example:\ndocker exec -it <container-id> <command>"